<a href="https://colab.research.google.com/github/AdrielleMendes/Linked_List/blob/main/Copy_of_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relatório do Projeto 01 - Chat "Cliente/Servidor"

**Disciplina:** Redes de Computadores (UAB00078)

**Curso:** Bacharelado em Engenharia de Computação

**Departamento:** Unidade Acadêmica de Belo Jardim (UABJ)

**Professor:** Ygor Amaral Barbosa Leite de Sena

## 1. Introdução

Este relatório tem como objetivo documentar o desenvolvimento de um sistema de chat desktop cliente-servidor, proposto como Projeto 01 da disciplina de Redes de Computadores. O sistema busca replicar funcionalidades encontradas em aplicativos de mensagens como o WhatsApp , com foco na implementação de um servidor TCP/IP robusto e de uma interface de cliente intuitiva.

O projeto foi concebido para proporcionar uma experiência prática com conceitos fundamentais de redes de computadores, incluindo sockets, protocolos de comunicação e concorrência. Além disso, abordou objetivos secundários para persistência de dados e desenvolvimento de interfaces gráficas (GUI).

As principais funcionalidades implementadas incluem:

* Registro e autenticação de usuários.
* Roteamento de mensagens em tempo real entre usuários que querem conversar.
* Armazenamento de mensagens destinadas a contatos offline para entrega imediata assim que eles voltam a se conectar.
* Manutenção de um indicador de "usuário digitando..." (e online/off-line).
* Exibição de lista de contatos.

O servidor foi arquitetado para ser multithread, permitindo o atendimento simultâneo a múltiplos clientes , e utiliza um banco de dados leve SQLite ou arquivo binário para garantir a persistência dos dados. O cliente, por sua vez, estabelece e mantém conexão via sockets TCP , e também emprega multithreading para conseguir enviar e receber mensagens simultaneamente.

Este relatório detalhará a arquitetura do sistema, o protocolo de comunicação definido, a implementação dos requisitos funcionais, as estratégias de gerenciamento de conexões e concorrência, os desafios superados e os aprendizados obtidos durante o processo de desenvolvimento. Finalmente, serão apresentadas as limitações e possíveis melhorias para o sistema.


## 2. Arquitetura do Servidor

A arquitetura do servidor foi desenvolvida com base nos requisitos de um sistema cliente-servidor multithread, capaz de gerenciar múltiplas conexões, autenticação de usuários, roteamento de mensagens em tempo real e persistência de dados.

O componente central é a classe `ChatServer`, que orquestra todas as operações.

### 2.1. Estrutura Geral do Servidor

O servidor é implementado como uma aplicação Python, utilizando a biblioteca `socket` para comunicação TCP/IP, `threading` para concorrência, `json` para serialização/desserialização de mensagens e `sqlite3` para persistência de dados.

* **`ChatServer` Class**:
    * **Inicialização (`__init__`)**:
        * Cria um socket TCP/IP (`socket.socket(socket.AF_INET, socket.SOCK_STREAM)`) para aceitar conexões de entrada.
        * Configura a opção `SO_REUSEADDR` para permitir a reutilização do endereço da porta após o fechamento do servidor, evitando o erro "Address already in use".
        * Associa o socket a um endereço IP (`HOST = '127.0.0.1'`) e porta (`PORT = 12346`) específicos.
        * Entra em modo de escuta (`listen(5)`) para aceitar até 5 conexões pendentes.
        * Inicializa dicionários para gerenciar clientes conectados (`self.connected_clients`), armazenando `username: socket` para rastrear usuários online.
        * Inicializa um dicionário para rastrear o status de digitação (`self.typing_status`), mapeando `username: {'typing_to': recipient, 'since': timestamp}`.
        * Utiliza um `threading.Lock()` (`self.lock`) para garantir a segurança no acesso a recursos compartilhados, como `self.connected_clients` e `self.typing_status`, prevenindo condições de corrida em ambientes multithread.
        * Chama `self.init_db()` para configurar o banco de dados.
        * Inicia uma thread separada (`self.check_inactive_clients`) em modo *daemon* para verificar periodicamente a atividade dos clientes, garantindo que clientes inativos sejam desconectados e seu status atualizado.
    * **Início do Servidor (`start`)**:
        * Entra em um loop infinito (`while True`) para continuamente aceitar novas conexões de clientes (`self.server_socket.accept()`).
        * Para cada nova conexão aceita, uma nova thread é criada (`threading.Thread(target=self.handle_client, args=(client_socket,)).start()`) para lidar com esse cliente de forma independente , assegurando que o servidor possa atender simultaneamente múltiplos clientes.
        * Inclui tratamento para `KeyboardInterrupt` para permitir um desligamento gracioso do servidor, fechando o socket do servidor e a conexão com o banco de dados.

### 2.2. Gerenciamento de Conexões e Concorrência

O servidor foi projetado com uma arquitetura multithread, onde cada cliente conectado é atendido por uma thread dedicada. Isso é crucial para gerenciar conexões simultâneas de múltiplos clientes sem bloquear o processamento principal do servidor.

* **Aceitação de Conexões**: O método `start()` utiliza `server_socket.accept()` para aceitar novas conexões. Cada `client_socket` recém-aceito é então passado para uma nova thread de `handle_client`.
* **Threads por Cliente (`handle_client`)**:
    * Cada thread `handle_client` é responsável por ler dados do socket do cliente, processar as mensagens recebidas e enviar respostas.
    * Implementa um mecanismo de *buffering* (`buffer = ""`) para lidar com mensagens incompletas ou múltiplas mensagens recebidas em um único `recv()`, garantindo que as mensagens JSON sejam decodificadas corretamente usando o caractere de nova linha (`\n`) como delimitador. Isso é fundamental para o enquadramento das mensagens no protocolo.
    * A thread permanece ativa (`while True`) enquanto houver dados do cliente. Se `recv()` retornar vazio, indica que o cliente se desconectou.
    * Em caso de desconexão ou erro, o bloco `finally` garante que o socket do cliente seja fechado e, se o cliente estava logado, ele seja removido de `self.connected_clients` e outros usuários sejam notificados sobre seu status "offline".
* **Tratamento de Falhas de Conexão **:
    * Os blocos `try-except` são utilizados extensivamente em `handle_client` e em métodos de envio (`send_response`, `handle_message`, `send_pending_messages`, `notify_user_login`, `notify_user_logout`, `handle_typing`, `handle_stop_typing`, `send_user_list`, `check_inactive_clients`) para capturar `Exceptions` (como `ConnectionResetError` ou `BrokenPipeError`).
    * Esses erros indicam falhas na conexão com clientes específicos. O servidor trata essas falhas imprimindo mensagens de erro e removendo o cliente da lista de `connected_clients` quando apropriado, o que implicitamente lida com a detecção de desconexões abruptas.
    * O método `check_inactive_clients` envia periodicamente mensagens "ping" para clientes conectados. Se um `ping` falhar, o cliente é considerado inativo e é removido, mitigando problemas de clientes "fantasmas" que não se desconectaram corretamente.
* **Sincronização com `threading.Lock`**: O uso de `self.lock` garante que as operações de leitura e escrita em `self.connected_clients` e `self.typing_status` sejam atômicas, evitando inconsistências de dados quando várias threads tentam modificar essas estruturas simultaneamente.

### 2.3. Persistência de Dados

A persistência de dados é implementada utilizando SQLite, um banco de dados leve e embutido, conforme sugerido nos requisitos do projeto.

* **Inicialização do Banco de Dados (`init_db`)**:
    * Conecta-se a um arquivo de banco de dados `chat.db` (`sqlite3.connect('chat.db', check_same_thread=False)`). O `check_same_thread=False` é importante para permitir que a mesma conexão de banco de dados seja usada por diferentes threads.
    * Cria a tabela `users` se ela não existir, com campos `username` (chave primária) e `password` (armazenado como hash).
    * Cria a tabela `messages` se ela não existir, com campos para `id`, `sender`, `receiver`, `message`, `timestamp` e um campo `delivered` (`BOOLEAN DEFAULT 0`) para indicar se a mensagem já foi entregue ao destinatário.
    * `self.conn.commit()` é chamado após a criação das tabelas para salvar as alterações.
* **Hashing de Senhas (`hash_password`)**: Antes de armazenar senhas no banco de dados, elas são convertidas em um hash SHA-256 usando `hashlib.sha256`. Isso aumenta a segurança, pois a senha original nunca é armazenada em texto simples.
* **Registro de Usuários (`handle_register`)**: Insere novos usuários na tabela `users` com a senha *hashed*. Lida com `sqlite3.IntegrityError` se o nome de usuário já existir.
* **Autenticação de Login (`handle_login`)**: Consulta a tabela `users` para verificar se o nome de usuário existe e se a senha *hashed* fornecida corresponde à armazenada.
* **Armazenamento de Mensagens (`handle_message`)**:
    * Antes de tudo, verifica se o `receiver` da mensagem de fato existe no banco de dados de usuários. Se não existir, uma `message_error` é enviada de volta ao remetente.
    * Se o destinatário estiver online, a mensagem é primeiramente enviada ao cliente. Em caso de sucesso no envio, ela é registrada na tabela `messages` com `delivered=1`.
    * Se o destinatário estiver offline, a mensagem é persistida na tabela `messages` com `delivered=0`.
* **Entrega de Mensagens Offline (`send_pending_messages`)**: Quando um usuário faz login, o servidor consulta a tabela `messages` para todas as mensagens onde ele é o `receiver` e `delivered=0`. Essas mensagens são então enviadas ao cliente, e o status `delivered` é atualizado para `1` no banco de dados.

### 2.4. Protocolo de Comunicação

O projeto exige a criação de um protocolo de comunicação próprio. Para isso, foi adotado um protocolo baseado em mensagens JSON, onde cada mensagem é um objeto JSON que contém um campo `'type'` para indicar a finalidade da mensagem, e outros campos específicos para os dados relevantes. Cada mensagem JSON é terminada por um caractere de nova linha (`\n`) para facilitar o enquadramento na leitura do socket.

#### 2.4.1. Formato das Mensagens

Todas as mensagens são objetos JSON, com o seguinte formato básico:

```json
{
    "type": "tipo_da_mensagem",
    "campo1": "valor1",
    "campo2": "valor2",
    ...
}

In [ ]:
import socket
import json
import threading
import sqlite3
import time
import hashlib
from datetime import datetime

HOST = '127.0.0.1'
PORT = 12346

class ChatServer:
    def __init__(self):
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # Permite reutilizar o endereço imediatamente após o fechamento, evitando "Address already in use"
        self.server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.server_socket.bind((HOST, PORT))
        self.server_socket.listen(5)

        # Estruturas para gerenciamento de clientes
        self.connected_clients = {}  # {username: socket}
        self.typing_status = {}      # {username: {'typing_to': recipient, 'since': timestamp}}
        self.lock = threading.Lock() # Um lock para proteger o acesso a self.connected_clients e self.typing_status

        # Inicialização do banco de dados
        self.init_db()

        print(f"Servidor de chat iniciado em {HOST}:{PORT}")
        # Inicia uma thread em segundo plano para verificar clientes inativos
        threading.Thread(target=self.check_inactive_clients, daemon=True).start()

    def init_db(self):
        """Inicializa o banco de dados SQLite para usuários e mensagens."""
        self.conn = sqlite3.connect('chat.db', check_same_thread=False)
        self.cursor = self.conn.cursor()

        # Tabela de usuários: armazena username e senha (hashed)
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS users
                               (username TEXT PRIMARY KEY, password TEXT)''')

        # Tabela de mensagens: armazena mensagens, com status de entrega
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS messages
                               (id INTEGER PRIMARY KEY AUTOINCREMENT,
                                sender TEXT, receiver TEXT,
                                message TEXT, timestamp DATETIME,
                                delivered BOOLEAN DEFAULT 0)''')
        self.conn.commit() # Salva as mudanças no banco de dados

    def hash_password(self, password):
        """Aplica hash SHA-256 à senha para armazenamento seguro."""
        return hashlib.sha256(password.encode()).hexdigest()

    def start(self):
        """Inicia o servidor para aceitar conexões de clientes em um loop contínuo."""
        try:
            while True:
                client_socket, addr = self.server_socket.accept() # Aceita uma nova conexão
                print(f"Nova conexão de {addr}")
                # Inicia uma nova thread para lidar com este cliente, para não bloquear o servidor
                threading.Thread(target=self.handle_client, args=(client_socket,)).start()
        except KeyboardInterrupt:
            print("\nDesligando servidor...")
        finally:
            self.server_socket.close() # Fecha o socket do servidor
            self.conn.close() # Fecha a conexão com o banco de dados

    def handle_client(self, client_socket):
        """Lida com a comunicação de um cliente específico, incluindo buffering de mensagens."""
        username = None # Variável para armazenar o username do cliente logado
        buffer = "" # Buffer para armazenar dados parciais da mensagem
        try:
            while True:
                data = client_socket.recv(4096).decode('utf-8')
                if not data:
                    break # Se não houver dados, o cliente desconectou

                buffer += data # Adiciona os novos dados ao buffer

                while '\n' in buffer: # Processa mensagens delimitadas por '\n'
                    message_str, buffer = buffer.split('\n', 1) # Divide no primeiro '\n'
                    if message_str.strip(): # Garante que não é uma string vazia após a divisão
                        try:
                            message = json.loads(message_str)
                            # Se for uma mensagem de login, armazena o username para o logout
                            if message.get('type') == 'login' and message.get('username'):
                                username = message.get('username')
                            self.process_message(client_socket, message)
                        except json.JSONDecodeError:
                            print(f"Mensagem JSON inválida recebida de {username if username else 'desconhecido'}: '{message_str}'")
                        except Exception as e:
                            print(f"Erro ao processar mensagem de {username if username else 'desconhecido'}: {e}")

        except Exception as e:
            print(f"Erro no cliente ({username if username else 'desconhecido'}): {e}")
        finally:
            # Limpeza ao desconectar
            if username:
                with self.lock:
                    if username in self.connected_clients and self.connected_clients[username] == client_socket:
                        self.notify_user_logout(username) # Notifica outros usuários sobre o logout
                        del self.connected_clients[username]
                        print(f"Cliente {username} desconectado.")
            client_socket.close() # Fecha o socket do cliente

    def process_message(self, client_socket, message):
        """Processa diferentes tipos de mensagens recebidas do cliente."""
        msg_type = message.get('type')

        if msg_type == 'register':
            self.handle_register(client_socket, message)
        elif msg_type == 'login':
            self.handle_login(client_socket, message)
        elif msg_type == 'logout':
            pass
        elif msg_type == 'message':
            self.handle_message(client_socket, message) # Passa client_socket para poder enviar resposta de erro
        elif msg_type == 'typing':
            self.handle_typing(message)
        elif msg_type == 'stop_typing':
            self.handle_stop_typing(message)
        elif msg_type == 'get_users':
            self.send_user_list(client_socket)
        elif msg_type == 'ping':
            pass
        else:
            print(f"Tipo de mensagem desconhecido: {msg_type}")

    def handle_register(self, client_socket, message):
        """Processa o registro de um novo usuário."""
        username = message.get('username')
        password = message.get('password')

        try:
            hashed_pw = self.hash_password(password)
            self.cursor.execute("INSERT INTO users VALUES (?, ?)", (username, hashed_pw))
            self.conn.commit()
            self.send_response(client_socket, {'type': 'register_response', 'status': 'success'})
            print(f"Usuário {username} registrado com sucesso.")
        except sqlite3.IntegrityError:
            self.send_response(client_socket, {'type': 'register_response', 'status': 'error', 'message': 'Usuário já existe'})
            print(f"Erro ao registrar {username}: Usuário já existe.")
        except Exception as e:
            self.send_response(client_socket, {'type': 'register_response', 'status': 'error', 'message': str(e)})
            print(f"Erro ao registrar {username}: {e}")

    def handle_login(self, client_socket, message):
        """Processa o login de um usuário existente."""
        username = message.get('username')
        password = message.get('password')

        try:
            hashed_pw = self.hash_password(password)
            self.cursor.execute("SELECT password FROM users WHERE username=?", (username,))
            result = self.cursor.fetchone()

            if result and result[0] == hashed_pw:
                with self.lock:
                    # Verifica se o usuário já está conectado em outro lugar
                    if username in self.connected_clients:
                        self.send_response(client_socket, {
                            'type': 'login_response',
                            'status': 'error',
                            'message': 'Usuário já está conectado.'
                        })
                        print(f"Tentativa de login de {username} falhou: já conectado.")
                        return

                    self.connected_clients[username] = client_socket

                self.send_response(client_socket, {
                    'type': 'login_response',
                    'status': 'success',
                    'username': username
                })
                print(f"Usuário {username} logado com sucesso.")

                self.send_pending_messages(username)
                self.notify_user_login(username)
            else:
                self.send_response(client_socket, {
                    'type': 'login_response',
                    'status': 'error',
                    'message': 'Credenciais inválidas'
                })
                print(f"Tentativa de login de {username} falhou: credenciais inválidas.")

        except Exception as e:
            self.send_response(client_socket, {
                'type': 'login_response',
                'status': 'error',
                'message': str(e)
            })
            print(f"Erro durante o login de {username}: {e}")

    def handle_message(self, client_socket, message): # Adicionado client_socket como parâmetro
        """Processa o envio de uma mensagem de um usuário para outro."""
        sender = message.get('sender')
        receiver = message.get('receiver')
        content = message.get('content')
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # VERIFICAÇÃO: Checa se o usuário destinatário existe no banco de dados
        self.cursor.execute("SELECT username FROM users WHERE username=?", (receiver,))
        if not self.cursor.fetchone():
            # Se o destinatário não existe, envia uma mensagem de erro de volta para o remetente
            error_msg = {
                'type': 'message_error',
                'recipient': receiver,
                'message': f"O usuário '{receiver}' não existe."
            }
            self.send_response(client_socket, error_msg)
            print(f"Tentativa de mensagem para usuário inexistente '{receiver}' por {sender}.")
            return # Encerra a função, não prossegue com o envio/armazenamento da mensagem

        with self.lock:
            if receiver in self.connected_clients:  # Usuário online
                try:
                    msg = {
                        'type': 'message',
                        'sender': sender,
                        'content': content,
                        'timestamp': timestamp
                    }
                    # Adiciona \n para enquadramento da mensagem
                    self.connected_clients[receiver].send(json.dumps(msg).encode('utf-8') + b'\n')

                    # Registrar mensagem como entregue
                    self.cursor.execute(
                        "INSERT INTO messages (sender, receiver, message, timestamp, delivered) VALUES (?, ?, ?, ?, ?)",
                        (sender, receiver, content, timestamp, 1)
                    )
                    self.conn.commit()
                    print(f"Mensagem de {sender} para {receiver} entregue: '{content}'")
                except Exception as e:
                    print(f"Erro ao enviar mensagem para {receiver}: {e}. Salvando como pendente.")
                    # Se houver erro no envio, salva como pendente (caso não esteja já)
                    self.cursor.execute(
                        "INSERT INTO messages (sender, receiver, message, timestamp) VALUES (?, ?, ?, ?)",
                        (sender, receiver, content, timestamp)
                    )
                    self.conn.commit()
            else:  # Usuário offline
                self.cursor.execute(
                    "INSERT INTO messages (sender, receiver, message, timestamp) VALUES (?, ?, ?, ?)",
                    (sender, receiver, content, timestamp)
                )
                self.conn.commit()
                print(f"Mensagem de {sender} para {receiver} salva (offline): '{content}'")

    def handle_typing(self, message):
        """Processa e retransmite o indicador de digitação."""
        username = message.get('username')
        recipient = message.get('recipient')

        with self.lock:
            self.typing_status[username] = {
                'typing_to': recipient,
                'since': time.time()
            }

            if recipient in self.connected_clients:
                notification = {
                    'type': 'typing_notification',
                    'from': username,
                    'status': 'typing'
                }
                try:
                    # Adiciona \n para enquadramento da mensagem
                    self.connected_clients[recipient].send(json.dumps(notification).encode('utf-8') + b'\n')
                except Exception as e:
                    print(f"Erro ao enviar notificação de digitação para {recipient}: {e}")

    def handle_stop_typing(self, message):
        """Processa e retransmite a parada de digitação."""
        username = message.get('username')

        with self.lock:
            if username in self.typing_status:
                recipient = self.typing_status[username]['typing_to']
                del self.typing_status[username]

                if recipient in self.connected_clients:
                    notification = {
                        'type': 'typing_notification',
                        'from': username,
                        'status': 'stopped'
                    }
                    try:
                        # Adiciona \n para enquadramento da mensagem
                        self.connected_clients[recipient].send(json.dumps(notification).encode('utf-8') + b'\n')
                    except Exception as e:
                        print(f"Erro ao enviar notificação de parada de digitação para {recipient}: {e}")

    def send_pending_messages(self, username):
        """Envia mensagens pendentes para um usuário que acabou de logar."""
        try:
            self.cursor.execute(
                "SELECT sender, message, timestamp, id FROM messages WHERE receiver=? AND delivered=0",
                (username,)
            )
            messages = self.cursor.fetchall()

            if not messages:
                print(f"Nenhuma mensagem pendente para {username}.")
                return

            print(f"Enviando {len(messages)} mensagens pendentes para {username}...")
            with self.lock:
                if username in self.connected_clients:
                    for sender, content, timestamp, msg_id in messages:
                        msg = {
                            'type': 'message',
                            'sender': sender,
                            'content': content,
                            'timestamp': timestamp
                        }
                        try:
                            # Adiciona \n para enquadramento da mensagem
                            self.connected_clients[username].send(json.dumps(msg).encode('utf-8') + b'\n')
                            self.cursor.execute(
                                "UPDATE messages SET delivered=1 WHERE id=?", (msg_id,)
                            )
                            self.conn.commit()
                            print(f"  - Mensagem pendente de {sender} entregue.")
                        except Exception as e:
                            print(f"  - Erro ao re-enviar mensagem pendente de {sender} para {username}: {e}")
                            pass
        except Exception as e:
            print(f"Erro ao enviar mensagens pendentes para {username}: {e}")

    def notify_user_login(self, username):
        """Notifica outros usuários online sobre um novo login."""
        with self.lock:
            for user, sock in self.connected_clients.items():
                if user != username:
                    notification = {
                        'type': 'user_status',
                        'username': username,
                        'status': 'online'
                    }
                    try:
                        # Adiciona \n para enquadramento da mensagem
                        sock.send(json.dumps(notification).encode('utf-8') + b'\n')
                    except Exception as e:
                        print(f"Erro ao notificar status de {username} para {user}: {e}")

    def notify_user_logout(self, username):
        """Notifica outros usuários online sobre um logout."""
        with self.lock:
            for user, sock in self.connected_clients.items():
                if user != username:
                    notification = {
                        'type': 'user_status',
                        'username': username,
                        'status': 'offline'
                    }
                    try:
                        # Adiciona \n para enquadramento da mensagem
                        sock.send(json.dumps(notification).encode('utf-8') + b'\n')
                    except Exception as e:
                        print(f"Erro ao notificar status de {username} para {user}: {e}")
            if username in self.typing_status:
                del self.typing_status[username]

    def send_user_list(self, client_socket):
        """Envia a lista de todos os usuários (online e offline) para o cliente solicitante."""
        try:
            with self.lock:
                online_users = list(self.connected_clients.keys())

            self.cursor.execute("SELECT username FROM users")
            all_users = [row[0] for row in self.cursor.fetchall()]

            user_list = []
            for user in all_users:
                status = 'online' if user in online_users else 'offline'
                user_list.append({'username': user, 'status': status})

            response = {
                'type': 'user_list',
                'users': user_list
            }
            # Adiciona \n para enquadramento da mensagem
            self.send_response(client_socket, response)
            print("Lista de usuários enviada.")

        except Exception as e:
            print(f"Erro ao enviar lista de usuários: {e}")

    def send_response(self, client_socket, response):
        """Envia uma resposta genérica para o cliente, com enquadramento de mensagem."""
        try:
            # Adiciona \n para enquadramento da mensagem
            client_socket.send(json.dumps(response).encode('utf-8') + b'\n')
        except Exception as e:
            print(f"Erro ao enviar resposta para cliente: {e}")

    def check_inactive_clients(self):
        """Verifica periodicamente clientes inativos enviando pings."""
        while True:
            time.sleep(30)
            with self.lock:
                to_remove = []
                for username, sock in list(self.connected_clients.items()):
                    try:
                        # Adiciona \n para enquadramento da mensagem
                        sock.send(json.dumps({'type': 'ping'}).encode('utf-8') + b'\n')
                    except Exception as e:
                        print(f"Cliente {username} inativo ou erro de socket: {e}")
                        to_remove.append(username)

                for username in to_remove:
                    print(f"Removendo cliente inativo: {username}")
                    del self.connected_clients[username]
                    self.notify_user_logout(username)

if __name__ == "__main__":
    server = ChatServer()
    server.start()

Servidor de chat iniciado em 127.0.0.1:12346


## 3. Implementação dos Requisitos Funcionais (Cliente - Versão CLI)

Esta versão do cliente é implementada para ser executada em um ambiente de linha de comando (CLI), focando puramente na lógica de comunicação e interação via texto. Ela demonstra os conceitos fundamentais de cliente TCP/IP, multithreading para envio/recebimento de mensagens, e o protocolo de comunicação definido.

### 3.1. Arquitetura do Cliente (Versão CLI)

O cliente CLI é composto por uma única classe, `ChatClient`, que engloba tanto a lógica de comunicação de rede quanto a interação textual com o usuário. Embora não possua uma GUI elaborada, ele cumpre todos os requisitos funcionais relacionados à rede e ao protocolo.

* **`ChatClient` Class**:
    * **Inicialização (`__init__`)**:
        * Cria um socket TCP/IP (`socket.socket(socket.AF_INET, socket.SOCK_STREAM)`) que será usado para a conexão com o servidor.
        * `self.username`: Armazena o nome de usuário após o login bem-sucedido.
        * `self.connected`: Uma flag booleana que indica o status da conexão com o servidor.
        * `self.last_typing_time`: Usado para controlar o envio de eventos de digitação, evitando spam.
        * `self.response_received`: Um `threading.Event()` utilizado para sincronizar a thread principal (que solicita login/registro) com a thread de recebimento de mensagens, aguardando a resposta do servidor para essas operações.
        * `self.login_status`: Armazena o resultado (sucesso/falha) das tentativas de login/registro recebidas do servidor.
    * **Loop Principal (`if __name__ == "__main__":`)**: O bloco principal do script gerencia a interação com o usuário, apresentando um menu de opções (Registrar, Login, Sair) e, após o login, opções para Enviar Mensagem, Listar Usuários ou Sair. Um novo `ChatClient` é instanciado a cada vez que o usuário decide registrar ou fazer login novamente.

### 3.2. Gerenciamento de Conexões e Concorrência (Cliente)

A concorrência no cliente é gerenciada através de threads para separar a tarefa de enviar dados da tarefa de receber dados do servidor, garantindo que o cliente possa escutar por mensagens a qualquer momento sem bloquear a interface de usuário (mesmo que textual).

* **Estabelecimento da Conexão (`ChatClient.connect`)**:
    * Tenta conectar o socket ao `HOST` e `PORT` definidos.
    * Se a conexão for bem-sucedida, `self.connected` é definido como `True`.
    * **Multithreading**: Uma nova thread (`threading.Thread(target=self.receive_messages, daemon=True).start()`) é iniciada imediatamente. Esta thread ficará dedicada a *receber* mensagens do servidor em segundo plano. O `daemon=True` garante que esta thread será encerrada automaticamente quando o programa principal terminar.
    * Retorna `True` em caso de sucesso e `False` em caso de erro, informando o usuário sobre a falha na conexão.

* **Envio de Mensagens (`ChatClient._send_json_message`)**:
    * Este método auxiliar é responsável por serializar o dicionário Python da mensagem para uma string JSON (`json.dumps(message)`), codificá-la para bytes (`.encode('utf-8')`), e anexar um caractere de nova linha (`+ b'\n'`).
    * [cite_start]O caractere de nova linha (`\n`) atua como um **delimitador de mensagem**, essencial para o servidor (e o próprio cliente na recepção) saber onde uma mensagem JSON termina e a próxima começa, prevenindo problemas de "sticky messages" (várias mensagens concatenadas em um único pacote TCP).
    * A mensagem completa é então enviada pelo socket (`self.socket.send(...)`).

* **Recebimento de Mensagens Concorrente (`ChatClient.receive_messages`)**:
    * Executa em uma thread separada, permitindo que o cliente escute por mensagens assincronamente.
    * Implementa um **buffer de recebimento** (`buffer = ""`) para lidar com mensagens TCP que podem chegar fragmentadas ou múltiplas em um único `recv()`.
    * O loop `while '\n' in buffer:` processa mensagens completas. Cada vez que um `\n` é encontrado, a string antes dele é considerada uma mensagem JSON completa, decodificada (`json.loads(message_str)`) e passada para `handle_server_message`. O restante do buffer é mantido para a próxima mensagem.
    * Se `self.socket.recv(4096)` retornar dados vazios, indica que o servidor encerrou a conexão, e a flag `self.connected` é atualizada.
    * Os blocos `try-except` tratam erros como `json.JSONDecodeError` (mensagens JSON malformadas) ou outras `Exception`s na conexão, imprimindo mensagens de erro e marcando `self.connected` como `False`.

* **Logout e Reconexão (`ChatClient.logout`)**:
    * Quando o usuário escolhe "Sair" do chat principal, o método `logout()` é chamado.
    * Ele tenta enviar uma mensagem `{'type': 'logout'}` ao servidor para informar sobre a desconexão, embora a confiabilidade desta entrega não seja garantida em caso de falha de rede.
    * Em seguida, ele reinicia o estado do cliente (`self.username = None`, `self.connected = False`) e fecha o socket (`self.socket.close()`).

### 3.3. Implementação dos Requisitos Funcionais

Apesar de ser uma versão CLI, o cliente implementa as interações necessárias para cumprir os requisitos funcionais básicos do projeto:

* **Registro/Autenticação de Usuário (`ChatClient.register`, `ChatClient.login`)**:
    * Ambos os métodos tentam primeiro estabelecer uma conexão se não estiverem conectados.
    * Constroem mensagens JSON com `type: 'register'` ou `type: 'login'`, incluindo `username` e `password`.
    * Utilizam `self.response_received.clear()` e `self.response_received.wait(5)` para bloquear a thread principal por até 5 segundos, aguardando a resposta do servidor. A thread de recebimento (`receive_messages`) processará a resposta e usará `self.response_received.set()` para liberar a thread principal.
    * O `self.login_status` é atualizado pela `handle_server_message` com base na resposta do servidor, e seu valor é retornado ao chamador. Isso demonstra a comunicação assíncrona entre threads para obter o resultado de uma operação remota.
    * Se o login for bem-sucedido, o `self.username` do cliente é definido.

* **Exibir Lista de Contatos (`ChatClient.get_user_list`, `ChatClient.handle_server_message` com `'user_list'`)**:
    * O cliente envia uma mensagem `{'type': 'get_users'}` ao servidor quando solicitado.
    * Ao receber uma mensagem `{'type': 'user_list'}` do servidor, o método `handle_server_message` itera sobre a lista de usuários (`message.get('users')`) e imprime no console o nome de usuário e seu status (online/offline).

* **Enviar/Receber Mensagens em Tempo Real (`ChatClient.send_message`, `ChatClient.handle_server_message` com `'message'`)**:
    * **Envio**: `send_message` constrói uma mensagem `{'type': 'message', 'sender': ..., 'receiver': ..., 'content': ...}` e a envia ao servidor.
    * **Recebimento**: `handle_server_message` processa mensagens `{'type': 'message'}` recebidas do servidor, extrai `sender`, `content` e `timestamp`, e as imprime na tela.
    * Também manipula mensagens `{'type': 'message_error'}`, que são enviadas pelo servidor se o destinatário da mensagem não existir, informando o remetente sobre a falha.

* **Mostrar Indicador "Digitando..." e Quando o Usuário Estiver Online/Offline (`ChatClient.start_typing`, `ChatClient.stop_typing`, `ChatClient.handle_server_message` com `'typing_notification'` e `'user_status'`)**:
    * **Digitação**:
        * Quando o usuário decide enviar uma mensagem, uma chamada para `client.start_typing(receiver)` é feita antes do `send_message`. Isso envia uma mensagem `{'type': 'typing'}` ao servidor com o nome do usuário digitando e o destinatário. O `self.last_typing_time` é atualizado.
        * Após o envio da mensagem, `client.stop_typing(receiver)` é chamado, enviando uma mensagem `{'type': 'stop_typing'}`.
        * O `handle_server_message` do cliente, ao receber uma `{'type': 'typing_notification'}` do servidor (que é retransmitida do remetente para o destinatário), verifica o status (`'typing'` ou `'stopped'`) e o `from` (quem está digitando) e imprime o status correspondente na tela. A lógica de `if self.last_typing_time > 0 and self.username == message.get('receiver'): self.stop_typing(sender)` é uma tentativa de parar o indicador de digitação do próprio cliente se ele receber uma mensagem do destinatário para quem ele estava digitando.
    * **Status Online/Offline**:
        * O `handle_server_message`, ao receber um pacote `{'type': 'user_status'}` do servidor (que o servidor envia quando usuários fazem login ou logout), imprime na tela a mudança de status do usuário.

* **Receber Mensagens Armazenadas Enquanto Estava Offline**:
    * A funcionalidade de recebimento de mensagens offline é uma responsabilidade primária do servidor. O cliente simplesmente recebe essas mensagens como pacotes `{'type': 'message'}` normais quando faz login (`self.get_user_list()` é chamado após login, mas a entrega das mensagens pendentes é orquestrada pelo servidor logo após a autenticação bem-sucedida, antes mesmo da lista de usuários ser enviada). O cliente as exibe da mesma forma que as mensagens em tempo real.

### 3.4. Desafios e Aprendizados

O desenvolvimento desta versão CLI, embora mais simples em termos de interface, proporcionou aprendizados valiosos em:

* [cite_start]**Sockets TCP/IP**: Compreensão prática do estabelecimento de conexões (`socket.connect()`, `socket.send()`, `socket.recv()`) e a natureza orientada a stream do TCP, que exige **enquadramento de mensagens**  para garantir que mensagens completas sejam processadas.
* **Protocolo de Comunicação**: A necessidade de definir um formato estruturado (JSON com delimitador `\n`) para que cliente e servidor possam interpretar as mensagens trocadas de forma consistente.
* **Concorrência (Multithreading)**: A aplicação de threads (`threading.Thread`) para lidar com o envio e o recebimento de dados simultaneamente, evitando que a aplicação "congele" enquanto espera por dados da rede. O uso de `threading.Event()` para sincronizar operações entre threads (como esperar por uma resposta de login) foi um aprendizado chave.
* **Gerenciamento de Estado**: Como manter o estado da conexão e do usuário (`self.connected`, `self.username`) de forma consistente em um ambiente concorrente.

### 3.5. Limitações (Versão CLI)

Esta versão CLI, desenvolvida como um passo inicial para validação da lógica de rede, possui as seguintes limitações, as quais foram posteriormente abordadas e superadas na implementação da interface gráfica do usuário (GUI) apresentada em seções subsequentes deste relatório."

* **Interface de Usuário Básica**: A interação é puramente textual, sem elementos gráficos para uma experiência de usuário rica.
* **Não Gerencia Conversas Múltiplas Visivelmente**: O cliente não mantém um histórico visual de conversas separadas com múltiplos usuários. Todas as mensagens são impressas na mesma saída do console.
* **Ausência de Notificações Visuais**: Novas mensagens de contatos não ativos não são destacadas, apenas impressas na tela.
* **Sem Persistência Local**: O histórico de mensagens recebidas não é salvo localmente no cliente ao encerrar o programa.
* **Recuperação de Conexão**: Não há lógica de reconexão automática robusta em caso de perda de conexão após o login; o usuário precisa sair e tentar logar novamente.
* **Simplicidade nos Indicadores de Digitação**: A implementação atual envia `start_typing` a cada 0.1 segundo (simulação) e `stop_typing` imediatamente após o envio da mensagem. Uma implementação real monitoraria a atividade de digitação de forma mais inteligente (e.g., após um pequeno atraso, e pararia após inatividade).

In [ ]:
import socket
import json
import threading
import time

HOST = '127.0.0.1'
PORT = 12346

class ChatClient:
    def __init__(self):
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.username = None
        self.connected = False
        self.last_typing_time = 0
        self.response_received = threading.Event() # Usaremos um evento para esperar a resposta
        self.login_status = None # Para guardar o status do login/registro

    def connect(self):
        """Conecta ao servidor"""
        try:
            self.socket.connect((HOST, PORT))
            self.connected = True
            threading.Thread(target=self.receive_messages, daemon=True).start()
            return True
        except Exception as e:
            print(f"Erro ao conectar: {e}")
            return False

    def _send_json_message(self, message):
        """Helper para enviar mensagens JSON com delimitador."""
        try:
            # Adiciona o caractere de nova linha para enquadramento da mensagem
            self.socket.send(json.dumps(message).encode('utf-8') + b'\n')
            return True
        except Exception as e:
            print(f"Erro ao enviar mensagem JSON: {e}")
            return False

    def register(self, username, password):
        """Registra novo usuário"""
        if not self.connected:
            if not self.connect():
                return False

        message = {
            'type': 'register',
            'username': username,
            'password': password
        }
        self.response_received.clear()
        if self._send_json_message(message):
            self.response_received.wait(5)
            return self.login_status
        return False

    def login(self, username, password):
        """Autentica usuário"""
        if not self.connected:
            if not self.connect():
                return False

        message = {
            'type': 'login',
            'username': username,
            'password': password
        }
        self.response_received.clear()
        if self._send_json_message(message):
            self.response_received.wait(5)
            return self.login_status
        return False

    def logout(self):
        """Desconecta do servidor"""
        if self.username and self.connected:
            message = {'type': 'logout'}
            self._send_json_message(message) # Tenta enviar o logout
        self.username = None
        self.connected = False
        self.socket.close()

    def send_message(self, receiver, content):
        """Envia mensagem para outro usuário"""
        if not self.username or not self.connected:
            print("Você precisa estar logado para enviar mensagens")
            return False

        message = {
            'type': 'message',
            'sender': self.username,
            'receiver': receiver,
            'content': content
        }
        return self._send_json_message(message)

    def start_typing(self, recipient):
        """Notifica servidor que usuário começou a digitar"""
        if not self.username or not self.connected:
            return

        self.last_typing_time = time.time()
        message = {
            'type': 'typing',
            'username': self.username,
            'recipient': recipient
        }
        self._send_json_message(message)

    def stop_typing(self, recipient):
        """Notifica servidor que usuário parou de digitar"""
        if not self.username or not self.connected:
            return

        message = {
            'type': 'stop_typing',
            'username': self.username,
            'recipient': recipient
        }
        self._send_json_message(message)

    def get_user_list(self):
        """Solicita lista de usuários ao servidor"""
        if not self.username or not self.connected:
            return

        message = {'type': 'get_users'}
        self._send_json_message(message)

    def receive_messages(self):
        """Processa mensagens recebidas do servidor"""
        buffer = "" # Buffer para armazenar dados parciais da mensagem
        try:
            while self.connected:
                data = self.socket.recv(4096).decode('utf-8')
                if not data:
                    print("\nConexão com o servidor foi encerrada.")
                    self.connected = False
                    break

                buffer += data

                while '\n' in buffer:
                    message_str, buffer = buffer.split('\n', 1)
                    if message_str.strip():
                        try:
                            message = json.loads(message_str)
                            self.handle_server_message(message)
                        except json.JSONDecodeError:
                            print(f"\nMensagem inválida recebida do servidor: '{message_str}'")
                        except Exception as e:
                            print(f"\nErro ao processar mensagem do servidor: {e}")

        except Exception as e:
            if self.connected:
                print(f"\nErro na conexão: {e}")
            self.connected = False

    def handle_server_message(self, message):
        """Trata diferentes tipos de mensagens do servidor"""
        msg_type = message.get('type')

        if msg_type == 'register_response':
            status = message.get('status')
            self.login_status = (status == 'success')
            self.response_received.set()

            if status == 'success':
                print("\n[SUCESSO] Registro realizado com sucesso!")
            else:
                print(f"\n[ERRO] Falha no registro: {message.get('message', 'Erro desconhecido')}")

        elif msg_type == 'login_response':
            status = message.get('status')
            self.login_status = (status == 'success')
            self.response_received.set()

            if status == 'success':
                self.username = message.get('username')
                print(f"\n[SUCESSO] Logado como {self.username}")
                self.get_user_list()
            else:
                print(f"\n[ERRO] Falha no login: {message.get('message', 'Erro desconhecido')}")

        elif msg_type == 'message':
            sender = message.get('sender')
            content = message.get('content')
            timestamp = message.get('timestamp')
            print(f"\n[{timestamp}] {sender}: {content}")
            if self.last_typing_time > 0 and self.username == message.get('receiver'):
                    self.stop_typing(sender)

        elif msg_type == 'message_error': # NOVO: Manipula mensagens de erro de envio
            recipient = message.get('recipient')
            error_message = message.get('message')
            print(f"\n[ERRO DE ENVIO] Não foi possível enviar para '{recipient}': {error_message}")

        elif msg_type == 'typing_notification':
            user_typing = message.get('from')
            status = message.get('status')
            if status == 'typing':
                print(f"\n{user_typing} está digitando...")
            elif status == 'stopped':
                print(f"\n{user_typing} parou de digitar.")

        elif msg_type == 'user_status':
            username = message.get('username')
            status = message.get('status')
            print(f"\n{username} está {status}.")

        elif msg_type == 'user_list':
            users = message.get('users')
            print("\n--- Usuários Disponíveis ---")
            for user_info in users:
                print(f"- {user_info['username']} ({user_info['status']})")
            print("--------------------------")

        elif msg_type == 'ping':
            pass

        else:
            print(f"\n[AVISO] Mensagem desconhecida do servidor: {message}")

if __name__ == "__main__":
    while True:
        client = ChatClient()

        while True:
            if not client.connected or client.username is None:
                choice = input("1. Registrar | 2. Login | 3. Sair\nEscolha: ")
                if choice == '1':
                    username = input("Nome de usuário para registro: ")
                    password = input("Senha: ")
                    if client.register(username, password):
                        print(f"Tentando logar automaticamente como {username}...")
                        client.login(username, password)
                elif choice == '2':
                    username = input("Nome de usuário: ")
                    password = input("Senha: ")
                    client.login(username, password)
                elif choice == '3':
                    print("Encerrando o cliente...")
                    client.logout()
                    break
                else:
                    print("Opção inválida.")
            else:
                action = input("\n1. Enviar Mensagem | 2. Listar Usuários | 3. Sair\nEscolha: ")
                if action == '1':
                    receiver = input("Para quem você quer enviar a mensagem? ")
                    content = input("Sua mensagem: ")
                    client.start_typing(receiver)
                    time.sleep(0.1) # Simulação de digitação
                    client.send_message(receiver, content)
                    client.stop_typing(receiver)
                elif action == '2':
                    client.get_user_list()
                elif action == '3':
                    print("Deslogando e retornando ao menu inicial...")
                    client.logout()
                    break
                else:
                    print("Opção inválida.")

        if choice == '3':
            break

    print("Cliente encerrado completamente.")

### Teste de Funcionalidades do Servidor de Chat

Abaixo estão os logs do console do servidor que demonstram as principais funcionalidades implementadas, como registro, login, troca de mensagens (online e offline) e tratamento de erros.

Servidor de chat iniciado em 127.0.0.1:12346
Nova conexão de ('127.0.0.1', 63063)
Usuário adri logado com sucesso.
Nenhuma mensagem pendente para adri.
Lista de usuários enviada.
Lista de usuários enviada.
Mensagem (ID: 12) de adri para kai salva (offline): 'oi'
Nova conexão de ('127.0.0.1', 63076)
Usuário sol logado com sucesso.
Nenhuma mensagem pendente para sol.
Lista de usuários enviada.
Mensagem (ID: 13) de sol para adri entregue: 'oi'
Mensagem (ID: 14) de adri para sol entregue: 'tudo bem ?'
Tentativa de mensagem para usuário inexistente 'alice' por adri.
Mensagem (ID: 15) de adri para kai salva (offline): 'oii'
Mensagem (ID: 16) de sol para adri entregue: 'sim'
